All the code we have now is implemented with recursion but recursion call are not efficient
https://stackoverflow.com/questions/7537989/recursion-vs-for-loops-factorials-java
(call stack stuff blablabla calling functions is bad vs loops)